In [1]:
from gensim.models import AuthorTopicModel
from gensim.corpora import Dictionary, bleicorpus
from gensim import corpora
from tqdm import tqdm_notebook
from pprint import pprint



# 사용자간의 유사성을 평가하기 위한 measure를 사용하기 위해 불러오기
from gensim.matutils import hellinger
from gensim import matutils

import pandas as pd
import os

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# Author Topic Model 실행
NUM_TOPICS = 4

if not os.path.exists("dongnoun_model"):
    model = AuthorTopicModel(corpus = corpus, id2word = dictionary.id2token , num_topics = NUM_TOPICS,
                            author2doc = author2doc , passes = noun0)
    model.save("dongnoun_model")
else:
    model = AuthorTopicModel.load("dongnoun_model")

In [3]:
# 학습 결과 확인하기
model.show_topic(0)

[('인증', 0.01739123662487431),
 ('알람', 0.016008289295582866),
 ('계좌', 0.015825125058413263),
 ('뱅크', 0.012701070355468316),
 ('도움', 0.01175529699033745),
 ('카드', 0.010589066165320658),
 ('버전', 0.010122900178913416),
 ('이걸', 0.009795041910391341),
 ('휴대폰', 0.009177273806147006),
 ('조아', 0.009038601499930547)]

In [4]:
# 토픽 라벨 지정
topic_labels= ["Topic0","Topic1","Topic2","Topic3"]

In [5]:
# 토픽별로 단어 확인하기
for topics in model.show_topics(model.num_topics):
    print("Label : "+ topic_labels[topics[0]])
    words = ""
    for word , prob in model.show_topic(topics[0],topn=20):
        words = words + word +" "
    print("Words :",words)
    print()

Label : Topic0
Words : 인증 알람 계좌 뱅크 도움 카드 버전 이걸 휴대폰 조아 방법 터치 문자 사업 크기 금융 결제 등록 은행 수신 

Label : Topic1
Words : 카드 사용 발급 기존 불편 이전 등록 페이 신규 가능 삼성 선불카드 신청 금액 잔액 기능 예전 건지 하나 체크카드 

Label : Topic2
Words : 사용 충전 결제 확인 불편 금액 가맹점 업데이트 내역 오류 가능 잔액 캐시 카드 개선 연결 접속 이용 하나 실행 

Label : Topic3
Words : 사용 포인트 편리 혜택 부산 적립 감사 캐시 최고 지역 프로 금액 만족 기대 유용 구분 경제 시백 이벤트 현금 



In [6]:
## 평점별로 토픽 분포 확인하기
def show_authors(star):
    print("Star : ", star)
    print("Docs" , model.author2doc[star])
    print("\nTopic of Distribution : ")
    print([(topic_labels[topic[0]] , topic[1]) for topic in model[star]])

In [7]:
# 별점 5의 토픽분포 확인
show_authors(5)

Star :  5
Docs [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 15, 16, 17, 19, 20, 54, 55, 63, 73, 75, 81, 82, 83, 88, 89, 90, 93, 97, 98, 99, 105, 123, 124, 126, 128, 129, 130, 131, 132, 136, 137, 138, 142, 143, 144, 145, 146, 147, 149, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 166, 167, 168, 170, 171, 172, 175, 178, 179, 180, 181, 184, 186, 187, 191, 192, 198, 204, 206, 208, 210, 212, 214, 217, 221, 222, 223, 224, 225, 226, 227, 228, 230, 232, 235, 237, 238, 240, 241, 242, 244, 245, 246, 249, 250, 252, 253, 254, 256, 258, 262, 267, 268, 275, 276, 278, 280, 293, 304, 314, 315, 317, 321, 325, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 346, 347, 348, 349, 352, 353, 354, 355, 356, 357, 358, 359, 362, 363, 364, 365, 370, 371, 372, 373, 374, 375, 376, 377, 379, 380, 381, 383, 385, 387, 389, 390, 391, 393, 394, 395, 398, 400, 401, 402, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 418, 419, 420, 421, 422, 423, 

In [8]:
[model[author] for author in model.id2author.values()]

[[(0, 0.24279214539134888),
  (1, 0.3180762106019059),
  (2, 0.3182920092045578),
  (3, 0.12083963480218753)],
 [(1, 0.2899146415782792), (2, 0.609651009343101), (3, 0.0942937487507775)],
 [(0, 0.3163044221430441),
  (1, 0.31514343681579066),
  (2, 0.3575140139448875),
  (3, 0.011038127096277722)],
 [(1, 0.19467691812076507), (2, 0.3457166696268943), (3, 0.4513536836054744)],
 [(0, 0.3246000682667971),
  (1, 0.5707082888836112),
  (2, 0.015314202359868452),
  (3, 0.08937744048972308)]]

In [9]:
# Hellinger Distance를 이용하여 비슷한 토픽을 가진 사용자를 추정하는 함수.


# author-topic 분포 만들기
author_vecs = [model[author] for author in model.id2author.values()]
 
def similarity(vec1, vec2):
    '''Get similarity between two vectors'''
    dist = hellinger(matutils.sparse2full(vec1, model.num_topics),
                     matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    
    return sim
 
def get_sims(vec):
    '''Get similarity of vector to all authors.'''
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims
 
def get_table(star, top_n=10, smallest_author=1):
    '''
    Get table with similarities, author names, and author sizes.
    Return `top_n` authors as a dataframe.
    
    '''
    
    # 유사도 측정하기
    sims = get_sims(model[star])
    
 
    # 저자별 정보 정렬하기
    table = []
    for elem in enumerate(sims):
        author_star = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_star])
        
        table.append((author_star,sim , author_size))
        

    
    # 사용자 패턴 분석 결과를 Dataframe으로 만들기
    df =pd.DataFrame(table,columns = ["Author","Score","Size"])
    df = df.sort_values("Score",ascending = False)[:top_n]
    
    
    return df

In [10]:
# 별점별 대화 패턴 검증
get_table(5)

,Author,Score,Size
4,5,1.000000,1090
0,1,0.743143,1207
2,3,0.720773,218
3,4,0.615896,214
1,2,0.612059,142
